In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# =========================================================
# 1. Path Dataset
# =========================================================
train_dir = "dataset/train"
test_dir = "dataset/test"

img_size = (224, 224)
batch = 32
AUTOTUNE = tf.data.AUTOTUNE

# =========================================================
# 2. Load Dataset
# =========================================================
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch,
    label_mode="binary"
)

test_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch,
    label_mode="binary"
)

class_names = train_ds.class_names
print("Classes:", class_names)

# =========================================================
# 3. Prefetch + Cache
# =========================================================
train_ds = train_ds.cache().shuffle(2000).prefetch(AUTOTUNE)
test_ds = test_ds.cache().prefetch(AUTOTUNE)

# =========================================================
# 4. Data Augmentation
# =========================================================
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.15),
])

# =========================================================
# 5. Build EfficientNet-B0 Model
# =========================================================
base_model = keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet",
    pooling="avg"
)

base_model.trainable = False  # 👍 Freeze dulu biar cepat & stabil

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = base_model(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.summary()

# =========================================================
# 6. Compile Model
# =========================================================
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# =========================================================
# 7. Callbacks
# =========================================================
checkpoint = keras.callbacks.ModelCheckpoint(
    "model/best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)

earlystop = keras.callbacks.EarlyStopping(
    patience=5,
    monitor="val_loss",
    restore_best_weights=True
)

# =========================================================
# 8. Train Stage 1 – Frozen Base Model
# =========================================================
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=[checkpoint, earlystop]
)

# =========================================================
# 9. Stage 2 – Fine-tune EfficientNet (Optional, improves accuracy)
# =========================================================
print("\n🔧 Fine-tuning EfficientNet...")

base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # low LR for fine-tuning
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=[checkpoint]
)

# =========================================================
# 10. Save Final Model
# =========================================================
model.save("model/ai_imageclassifier_v3.keras")
print("\n✅ Training complete. Model saved!")

Found 971 files belonging to 2 classes.
Detected classes: ['AiArtData', 'RealArt']
Found 971 files belonging to 2 classes.
Using 680 files for training.
Found 971 files belonging to 2 classes.
Using 291 files for validation.


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,481,569 (17.10 MB)

 Trainable params: 4,481,569 (17.10 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 14s 497ms/step - accuracy: 0.5698 - loss: 0.6913 - val_accuracy: 0.5120 - val_loss: 0.6924 - learning_rate: 1.0000e-04
Epoch 2/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 484ms/step - accuracy: 0.5678 - loss: 0.6876 - val_accuracy: 0.5120 - val_loss: 0.6940 - learning_rate: 1.0000e-04
Epoch 3/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 473ms/step - accuracy: 0.5854 - loss: 0.6732 - val_accuracy: 0.5258 - val_loss: 0.6831 - learning_rate: 1.0000e-04
Epoch 4/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 477ms/step - accuracy: 0.6003 - loss: 0.6520 - val_accuracy: 0.5601 - val_loss: 0.6790 - learning_rate: 1.0000e-04
Epoch 5/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 483ms/step - accuracy: 0.5733 - loss: 0.6702 - val_accuracy: 0.5773 - val_loss: 0.6740 - learning_rate: 1.0000e-04
Epoch 6/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 487ms/step - accuracy: 0.6394 - loss: 0.6551 - val_accuracy: 0.5498 - val_loss: 0.6994 - learning_rate: 1.0000e-04
Epoch 7/30
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 497ms/step - acc